# Regression patterns
This notebook visualizes results from `SST_regression.py`.

- mean map and correlation to obs
- std of maps
- stationarity test: correlations of maps with their mean

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
from maps import regr_map
from paths import path_samoc, path_results, file_ex_ocn_ctrl
from timeseries import IterateOutputCESM
from ab_derivation_SST import DeriveSST as DS
from bc_analysis_fields import AnalyzeField as AF
from bd_analysis_indices import AnalyzeIndex as AI
from xr_regression import xr_quadtrend

## HAD Regression patterns in (detrended) observations

In [ ]:
for idx in ['AMO', 'SOM', 'TPI']:
    ds = xr.open_dataset(f'{path_samoc}/SST/{idx}_regr_had.nc')
    regr_map(ds=ds, index=idx, run='had', fn=f'{path_results}/SST/{idx}_regr_map_had')

### mean pattern for ctrl and lpd

In [ ]:
for j, run in enumerate(['ctrl', 'lpd']):
    if run=='ctrl':   starts = np.arange(50, 151, 5)
    elif run=='lpd':  starts = np.arange(154, 415, 10)
        
    for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
        for k, t in enumerate(starts):
            print(j,k,i)
            tslice = (t, t+148)
            fn = f'{path_samoc}/SST/{idx}_regr_{run}_{tslice[0]}_{tslice[1]}.nc'
            ds_temp = xr.open_dataset(fn, decode_times=False)
            ds_temp['start_year'] = tslice[0]
            ds_temp = ds_temp.expand_dims('start_year')
            ds_temp = ds_temp.set_coords('start_year', tslice[0])
            if k==0:
                ds = ds_temp
            else:
                ds = xr.concat([ds,ds_temp], dim='start_year')
        ds.to_netcdf(f'{path_samoc}/SST/{idx}_regr_{run}_all_segments.nc')
        ds.close()

In [ ]:
# calculating mean and std
for j, run in enumerate(['ctrl', 'lpd']):
    for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
        print(run, idx)
        fn = f'{path_samoc}/SST/{idx}_regr_{run}_all_segments.nc'
        da = xr.open_dataset(fn).slope
        fn_new = f'{path_samoc}/SST/{idx}_regr_{run}_all_segments_mean.nc'
        da.mean(dim='start_year').to_netcdf(fn_new)
        fn_new = f'{path_samoc}/SST/{idx}_regr_{run}_all_segments_std.nc'
        da.std(dim='start_year').to_netcdf(fn_new)

In [ ]:
from regions import boolean_mask, SST_index_bounds
from maps import make_map, rect_polygon
from plotting import discrete_cmap
from paths import file_ex_ocn_lpd
import cmocean

In [ ]:
# plotting mean
for j, run in enumerate(['ctrl', 'lpd']):
    if run in ['ctrl', 'rcp']:   
        domain = 'ocn'
    elif run in ['lpd', 'lpi']:  
        domain = 'ocn_low'
        TLAT = xr.open_dataset(file_ex_ocn_lpd, decode_times=False).TLAT.coords['TLAT']
        TLONG = xr.open_dataset(file_ex_ocn_lpd, decode_times=False).TLONG.coords['TLONG']
        
    MASK = boolean_mask(domain=domain, mask_nr=0)
    
    for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
        if idx in ['AMO', 'SOM']:
            rects = rect_polygon(SST_index_bounds(idx))
            clon = 300
            nv = .4
        elif idx=='TPI':
            rects = [rect_polygon(SST_index_bounds('TPI1')),
                     rect_polygon(SST_index_bounds('TPI2')),
                     rect_polygon(SST_index_bounds('TPI3')),
                    ]
            clon = 200
            nv = .3
            
        proj = 'rob'
        cm = discrete_cmap(16, cmocean.cm.balance)    
        label ='regression slope [K/K]'
        text1 = f'SST({idx})\nregr.'
        text2 = f'{run.upper()}\nmean'
        if run in ['ctrl', 'rcp']:   domain = 'ocn_T'

        # mean
        fn = f'{path_samoc}/SST/{idx}_regr_{run}_all_segments_mean.nc'
        xa = xr.open_dataarray(fn)
        if run=='lpd':
            xa.coords['TLAT'] = TLAT
            xa.coords['TLONG'] = TLONG
        xa = xa.where(MASK)

        fn_new = f'{path_results}/SST/{idx}_regr_map_{run}_all_segments_mean'
        f, ax = make_map(xa=xa, domain=domain, proj=proj, cmap=cm, minv=-nv, maxv=nv,
                     label=label, filename=fn_new, text1=text1, text2=text2,
                     rects=rects, clon=clon)
        
        # std
        text2 = f'{run.upper()}\nstd'
        nv = nv/5
        fn = f'{path_samoc}/SST/{idx}_regr_{run}_all_segments_std.nc'
        xa = xr.open_dataarray(fn)
        if run=='lpd':
            xa.coords['TLAT'] = TLAT
            xa.coords['TLONG'] = TLONG
        xa = xa.where(MASK)        
        
        fn_new = f'{path_results}/SST/{idx}_regr_map_{run}_all_segments_std'
        f, ax = make_map(xa=xa, domain=domain, proj=proj, cmap=cm, minv=-nv, maxv=nv,
                     label=label, filename=fn_new, text1=text1, text2=text2,
                     rects=rects, clon=clon)

### pattern correlation

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(8,3), sharey=True, constrained_layout=True)
for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
    for j, run in enumerate(['ctrl', 'lpd']):
        fn = f'{path_samoc}/SST/{idx}_spatial_correlations_{run}.nc'
        da = xr.open_dataarray(fn)
        da.plot(label=idx, ax=ax[j])
        ax[j].set_xlabel('starting year of segment')
        ax[j].text(da.time[0], .7, run.upper())
        ax[j].tick_params()
        ax[j].axhline(0, c='k', lw=.3)

ax[0].legend(ncol=3, loc=8)
ax[0].set_ylabel('spatial correlation coefficient')
plt.savefig(f'{path_results}/SST/spatial_correlation(t)_ctrl_lpd')

## Stationarity
spatial correlation of segments with mean

In [ ]:
from paths import file_ex_ocn_ctrl
from bc_analysis_fields import AnalyzeField as AF

In [ ]:
for j, run in enumerate(['ctrl', 'lpd']):
    if run=='ctrl':  TLAT = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).TLAT.coords['TLAT']

    for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
#         region = [{'latitude':slice(1081,1858), 'longitude':slice( 500,1100)}, 1, 2][i]
    
        if run=='ctrl':   starts = np.arange(50, 151, 5)
        elif run=='lpd':  starts = np.arange(154, 415, 10)
        
        da = xr.DataArray(data=np.zeros(len(starts)),
                          coords={'time': starts},
                          dims=('time'))
        
        
        
        fn = f'{path_samoc}/SST/{idx}_regr_{run}_all_segments_mean.nc'
        xa = xr.open_dataarray(fn)
        for k, t in enumerate(starts):
            print(i,j,k)
            tslice = (t, t+148)
            fn = f'{path_samoc}/SST/{idx}_regr_{run}_{tslice[0]}_{tslice[1]}.nc'
            xa_temp = xr.open_dataset(fn, decode_times=False).slope
            if run=='ctrl':
                xa_temp.coords['TLAT'] = TLAT
            
            da.values[k] = AF().spatial_correlation(field_A=xa, field_B=xa_temp, selection=region)
        da.to_netcdf(f'{path_samoc}/SST/{idx}_spatial_correlations_segments_to_mean_{run}.nc')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(8,3), sharey=True, constrained_layout=True)
for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
    for j, run in enumerate(['ctrl', 'lpd']):
        fn = f'{path_samoc}/SST/{idx}_spatial_correlations_segments_to_mean_{run}.nc'
        da = xr.open_dataarray(fn)
        da.plot(label=idx, ax=ax[j])
        ax[j].set_xlabel('starting year of segment')
        ax[j].text(da.time[0], .7, run.upper())
        ax[j].tick_params()
        ax[j].axhline(0, c='k', lw=.3)

ax[0].legend(ncol=3, loc=8)
ax[0].set_ylabel('spatial correlation coefficient')
plt.savefig(f'{path_results}/SST/spatial_correlation(t)_segments_to_mean_ctrl_lpd')